In [31]:
!pip install -q -U google-genai
!pip install pydantic
!pip install seqeval
!pip install google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 18.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 19.3 MB/s eta 0:00:00 0:00:01


In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("eriktks/conll2003")

# Access the train, validation, and test splits
train_data = dataset["train"]
validation_data = dataset["validation"]
test_data = dataset["test"]

# Print a sample
print(train_data[0])

/Users/francescostella/anaconda3/envs/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}


In [3]:
# Print a sample from the training data
example = train_data[11001]
print("Example from the training data:")
print(example)
print("Tokens:", example["tokens"])
print("Labels:", example["ner_tags"])

Example from the training data:
{'id': '11001', 'tokens': ['1886', '-', 'At', 'Skeleton', 'Canyon', 'in', 'Arizona', ',', 'Geronimo', ',', 'Apache', 'chief', 'and', 'leader', 'of', 'the', 'last', 'great', 'Red', 'Indian', 'rebellion', 'finally', 'surrendered', 'to', 'General', 'Nelson', 'Miles', '.'], 'pos_tags': [11, 8, 15, 22, 22, 15, 22, 6, 22, 6, 22, 21, 10, 21, 15, 12, 16, 16, 22, 22, 21, 30, 38, 35, 22, 22, 22, 7], 'chunk_tags': [11, 12, 13, 11, 12, 13, 11, 0, 11, 0, 11, 12, 0, 11, 13, 11, 12, 12, 12, 12, 12, 3, 21, 13, 11, 12, 12, 0], 'ner_tags': [0, 0, 0, 5, 6, 0, 5, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 1, 2, 0]}
Tokens: ['1886', '-', 'At', 'Skeleton', 'Canyon', 'in', 'Arizona', ',', 'Geronimo', ',', 'Apache', 'chief', 'and', 'leader', 'of', 'the', 'last', 'great', 'Red', 'Indian', 'rebellion', 'finally', 'surrendered', 'to', 'General', 'Nelson', 'Miles', '.']
Labels: [0, 0, 0, 5, 6, 0, 5, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 1, 2, 0]


In [4]:
print("Training set size:", len(train_data))
print("Validation set size:", len(validation_data))
print("Test set size:", len(test_data))

Training set size: 14041
Validation set size: 3250
Test set size: 3453


{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}

In [ ]:
import csv
import os


def parse_response(tokens : list, response_labels : list, true_labels : list) -> list: 
    '''
    Store the response in a list of lists where the first element is the token, the second element 
    is the predicted label and the third is the true label
    '''
    response_labels = response_labels.split(":")
    response_labels = response_labels[1].strip('\n').split(',')
    if (len(response_labels) != len(tokens)):
        if (len(response_labels) > len(tokens)):
            response_labels = response_labels[:len(tokens)]
        if (len(response_labels) < len(tokens)):
            response_labels = response_labels + ['0'] * (len(tokens) - len(response_labels))

    temp = []
    for i in range(len(tokens)):
        pred_label = int(response_labels[i].strip())
        
        assert (pred_label >= 0 and pred_label <= 8), "Predicted label is out of range"
        temp.append([tokens[i], pred_label, true_labels[i]])

    return temp

def save_to_csv_vanilla(tokens : list, pred_labels : list, true_labels : list, filename : str) -> None:
    file_exists = os.path.isfile(filename)
    # Write header only if the file didn't exist before
    if not file_exists:
        with open(filename, 'a', newline='') as csvfile:
            header = ['token', 'pred', 'true']
            writer = csv.writer(csvfile)
            writer.writerow(header)
    data = [[tokens[i], pred_labels[i], true_labels[i]] for i in range(len(tokens)) if pred_labels[i] != 0 and true_labels[i] != 0]
    # Remove duplicates
    # Open the file in append mode and write data to analysis purpose
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)

In [234]:
# Take predicted labels and for each token save the label in a list to be used for voting
def store_predicted_labels(pred_labels : list, votes : list) -> None:
    for i in range(len(pred_labels)):
        votes[i].append(pred_labels[i])

In [24]:
import random

# Sample 300 random elements from the test set
sampled_test_data = random.sample(list(test_data), 300)

# Print the first few samples to verify
for i, sample in enumerate(sampled_test_data[:5]):  # Display the first 5 samples
    print(f"Sample {i + 1}:")
    print(sample)
    print("\n")

Sample 1:
{'id': '1645', 'tokens': ['12/15/2007', '850M', '4.85', '4.85'], 'pos_tags': [11, 11, 11, 11], 'chunk_tags': [11, 12, 12, 12], 'ner_tags': [0, 0, 0, 0]}


Sample 2:
{'id': '986', 'tokens': ['Tambang', 'Timah', 'at', '$', '15.575', 'in', 'London', '.'], 'pos_tags': [39, 22, 15, 3, 11, 15, 22, 7], 'chunk_tags': [21, 11, 13, 11, 12, 13, 11, 0], 'ner_tags': [3, 4, 0, 0, 0, 0, 5, 0]}


Sample 3:
{'id': '529', 'tokens': ['SAN', 'FRANCISCO', '10', '3', '0', '325', '198'], 'pos_tags': [22, 22, 11, 11, 11, 11, 11], 'chunk_tags': [11, 12, 12, 12, 12, 12, 12], 'ner_tags': [3, 4, 0, 0, 0, 0, 0]}


Sample 4:
{'id': '302', 'tokens': ['The', 'West', 'Indies', 'vice-captain', 'struggled', 'for', 'timing', 'during', 'his', '36-minute', 'stay', 'at', 'the', 'crease', 'before', 'chipping', 'a', 'ball', 'from', 'medium', 'pacer', 'Tom', 'Moody', 'straight', 'to', 'Shane', 'Warne', 'at', 'mid-wicket', '.'], 'pos_tags': [12, 22, 22, 16, 38, 15, 21, 15, 29, 16, 21, 15, 12, 21, 15, 39, 12, 21, 15, 2

### Vanilla Method

In [ ]:
from google import genai

for j in range(len(sampled_test_data)):
    # Extract tokens and true labels
    tokens = sampled_test_data[j]['tokens']
    true_labels = sampled_test_data[j]['ner_tags']
    model = "gemma-3-27b-it"
    client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
    # Send request 
    response = client.models.generate_content(
        model=model,
        contents=f"""Given the following NER tags: {{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}}, determine the Named Entity Recognition (NER) tags for the following sentence.
        The sentence is: '{tokens}'
        This sentence contains exactly {len(tokens)} tokens.

        Print only the number associated with the NER tag for each of the {len(tokens)} tokens, using the tag-to-number mapping provided above.
        Your answer MUST follow the format: ner_tags: 0, 1, 2, 0, 0, 0
        The number of output NER tags MUST be exactly {len(tokens)}, one for each token in the order they appear in the sentence.
        Do not include any other text or explanations.
        """
    )

    # Parse the response
    data = parse_response(tokens, response.text, true_labels)

    save_to_csv_vanilla(tokens, [item[1] for item in data], true_labels, "data/vanilla_test_300.csv")

In [ ]:
# import csv
# import os

# label_mapping = {
#     0: 'O',
#     1: 'B-PER',
#     2: 'I-PER',
#     3: 'B-ORG',
#     4: 'I-ORG',
#     5: 'B-LOC',
#     6: 'I-LOC',
#     7: 'B-MISC',
#     8: 'I-MISC'
# }

# def save_basic_decomposed_qa_to_csv(tokens: list, pred_labels: dict, true_labels: dict, filename: str) -> None:
#     """
#     Save the token, predicted labels, and true labels to a CSV file for Basic Decomposed-QA.

#     Args:
#         tokens (list): List of tokens in the sentence.
#         pred_labels (dict): Dictionary of predicted labels in the format {'PER': ['entity1', 'entity2'], ...}.
#         true_labels (dict): Dictionary of true labels in the format {'PER': ['entity1', 'entity2'], ...}.
#         filename (str): Path to the CSV file (relative to the 'data' folder).
#     """
#     # Ensure the 'data' directory exists
#     os.makedirs("data", exist_ok=True)
#     filepath = os.path.join("data", filename)

#     file_exists = os.path.isfile(filepath)

#     if not file_exists:
#         with open(filepath, 'w', newline='') as csvfile:
#             writer = csv.writer(csvfile)
#             writer.writerow(['token', 'pred', 'true'])

#     # Create reverse lookup from token to label index
#     def get_label_index(token_idx, label_dict):
#         for label_str, entities in label_dict.items():
#             if entities == 'None':
#                 continue
#             tokens_in_label = [t.strip() for t in entities.split(',')]
#             if tokens[token_idx] in tokens_in_label:
#                 # Convert label to index
#                 for idx, tag in label_mapping.items():
#                     if tag == label_str:
#                         return idx
#         return 0  # 'O'

#     # Write data
#     with open(filepath, 'a', newline='') as csvfile:
#         writer = csv.writer(csvfile)
#         for i, token in enumerate(tokens):
#             pred_idx = get_label_index(i, pred_labels)
#             true_idx = get_label_index(i, true_labels)
#             writer.writerow([token, pred_idx, true_idx])

In [ ]:
import csv
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

label_mapping = {
    0: 'O',
    1: 'B-PER',
    2: 'I-PER',
    3: 'B-ORG',
    4: 'I-ORG',
    5: 'B-LOC',
    6: 'I-LOC',
    7: 'B-MISC',
    8: 'I-MISC'
}

category_to_index = {
    'O': 0,
    'B-PER': 1, 
    'I-PER': 2, 
    'B-ORG': 3, 
    'I-ORG': 4, 
    'B-LOC': 5, 
    'I-LOC': 6, 
    'B-MISC': 7, 
    'I-MISC': 8
    }

# Step 2: Read the CSV and convert predictions and true labels
true_seqs = []
pred_seqs = []
current_true = []
current_pred = []

with open('data/vanilla_test_300.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        token = row['token']
        # true_label = index_to_label[int(row['true'])]
        # pred_label = index_to_label[int(row['pred'])]
        true_label = label_mapping[int(row['true'])]
        pred_label = label_mapping[int(row['pred'])]

        current_true.append(true_label)
        current_pred.append(pred_label)

    true_seqs.append(current_true)
    pred_seqs.append(current_pred)

# Step 3: Compute metrics
print("Precision:", precision_score(true_seqs, pred_seqs))
print("Recall:", recall_score(true_seqs, pred_seqs))
print("F1 Score:", f1_score(true_seqs, pred_seqs))

# Optional detailed report
print("\nDetailed classification report:\n")
print(classification_report(true_seqs, pred_seqs))


Precision: 0.5513626834381551
Recall: 0.6368038740920097
F1 Score: 0.5910112359550561

Detailed classification report:

              precision    recall  f1-score   support

         LOC       0.56      0.86      0.68       139
        MISC       0.43      0.29      0.35        41
         ORG       0.26      0.23      0.24       118
         PER       0.79      0.90      0.84       115

   micro avg       0.55      0.64      0.59       413
   macro avg       0.51      0.57      0.53       413
weighted avg       0.53      0.64      0.57       413



### Decomposed-QA

In [ ]:
#Decomposed-QA
import google.generativeai as genai
import os
import time
    
def save_to_csv_qa(pred_labels, true_labels_indices, tokens, filename):
    data = []
    
    # Convert true_labels_indices to a dictionary mapping tokens to their true labels
    true_label_dict = {}
    for token, label_idx in zip(tokens, true_labels_indices):
        true_label_dict[token] = label_mapping[label_idx]
    
    print("True label dictionary:", true_label_dict)
    print("Predicted labels:", pred_labels)
    
    # Process each predicted label and their tokens
    for pred_label, tokens_str in pred_labels.items():
        # Split and remove whitespace from the predicted tokens
        token_list = [t.strip() for t in tokens_str.split(',')]
        
        # Check if any of these tokens are in our original token list
        # If not, they might be composite entities that need special handling
        for token in token_list:
            if token in tokens:
                # Direct match with a single token
                true_label = true_label_dict.get(token, 'O')
                data.append([token, category_to_index[pred_label], category_to_index[true_label]])
            else:
                # This might be a composite token or doesn't exist in the original tokens
                # Try to match with individual tokens from the original list
                token_found = False
                for orig_token in tokens:
                    # Check if this token is part of the composite or has been recognized separately
                    if orig_token == token or token.find(orig_token) >= 0:
                        true_label = true_label_dict.get(orig_token, 'O')
                        data.append([orig_token, category_to_index[pred_label], category_to_index[true_label]])
                        token_found = True
                
                if not token_found:
                    # If we can't match it, add it with the predicted label but mark true as 'O'
                    print(f"Warning: Could not match predicted token '{token}' to original tokens")
                    data.append([token, category_to_index[pred_label], category_to_index['O']])
    
    # Process any tokens that weren't in the predictions but have true labels
    for token, true_label in true_label_dict.items():
        if true_label != 'O':
            # Check if this token was already processed
            token_processed = any(entry[0] == token for entry in data)
            if not token_processed:
                data.append([token, category_to_index['O'], category_to_index[true_label]])
    

    # Write to CSV
    file_exists = os.path.isfile(filename)
    if not file_exists:
        with open(filename, 'a', newline='') as csvfile:
            header = ['token', 'pred', 'true']
            writer = csv.writer(csvfile)
            writer.writerow(header)
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)

    


label_groups = [(1, 2), (3, 4), (5, 6), (7, 8)]

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel('gemini-2.5-flash-preview-04-17')  # or gemma-3-27b-it

for j in range(len(sampled_test_data)):
    tokens = sampled_test_data[j]['tokens']
    true_labels = sampled_test_data[j]['ner_tags']
    results = {}
    
    chat = model.start_chat()
    # Send general instructions only at the beginning of the conversation
    intro_message = (
        f"""Given the following NER tags: {{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}}, determine the Named Entity Recognition (NER) tags for the following sentence.
        If there are multiple tokens for a single category then separate the list of tokens with commas.
        
        The sentence is: '{tokens}'.
        This sentence contains exactly {len(tokens)} tokens.
        Remember that a token can only be classified once."""
    )
    # Send message to the chat
    try:
        chat.send_message(intro_message)
        time.sleep(5)
    except Exception as e:
        # If the request fails, try again after 30 seconds to avoid rate limits
        time.sleep(30)
        chat.send_message(intro_message)

    # Send a question for each label group in BIO format
    for group in label_groups:
        label_1 = label_mapping[group[0]]
        label_2 = label_mapping[group[1]]
        prompt = (
            f"""Which are the tokens labeled as '{label_1}' and '{label_2}' in the text? 
            If there are multiple tokens for a single category then separate the list of tokens with commas.
            The output must be in the format: 'B-PER: entity1, entity2, entity3' & 'I-PER: entity4, entity5'.
            If only one category is present, then the output should be: 'B-PER: entity1, entity2, entity3' or 'I-PER: entity4, entity5'.
            If both categories have no entities, just answer with 'None'."""
        )

        try:
            response = chat.send_message(prompt)
            time.sleep(5)
        except Exception as e:
            time.sleep(30)
            response = chat.send_message(prompt)


        if (response.text != 'None'):
            answers = response.text.split('&')
            answers = [answer.strip() for answer in answers]
            print("Answers:", answers)
            for answer in answers:
                answer = answer.strip()
                print("Answer:", answer)
                if answer != 'None':
                    # Check if the answar contains a colon
                    if ':' in answer:
                        # Handle the case where there might be nothing after the colon
                        # Split on the first colon only
                        parts = answer.split(':', 1)  
                        label = parts[0].strip()
                        
                        # Check if there's content after the colon
                        entities = parts[1].strip() if len(parts) > 1 and parts[1].strip() else ""
                        
                        # Only update results if we actually have entities
                        if entities:
                            results.update({label: entities})
                        else:
                            continue
                            #print(f"Warning: No entities found for label {label}")
                    else:
                        continue
                        #print(f"Warning: Invalid format in answer: {answer}")
            
    # Save result to CSV with ad-hoc functin for Basic Decomposed-QA
    save_to_csv_qa(results, true_labels, tokens, "data/qa_test_light.csv")

Answers: ['B-PER: Marshall, Jason', 'I-PER: Faulk, Belser']
Answer: B-PER: Marshall, Jason
Answer: I-PER: Faulk, Belser
Answers: ['B-ORG: Colts, Eagles', 'I-ORG:']
Answer: B-ORG: Colts, Eagles
Answer: I-ORG:
True label dictionary: {'Marshall': 'B-PER', 'Faulk': 'I-PER', 'rushed': 'O', 'for': 'O', '101': 'O', 'yards': 'O', 'and': 'O', 'two': 'O', 'touchdowns': 'O', 'Jason': 'B-PER', 'Belser': 'I-PER', 'returned': 'O', 'an': 'O', 'interception': 'O', '44': 'O', 'a': 'O', 'score': 'O', 'as': 'O', 'the': 'O', 'Colts': 'B-ORG', 'improved': 'O', 'to': 'O', '8-6': 'O', ',': 'O', 'same': 'O', 'mark': 'O', 'Eagles': 'B-ORG', 'who': 'O', 'lost': 'O', 'fourth': 'O', 'time': 'O', 'in': 'O', 'five': 'O', 'games': 'O', '.': 'O'}
Predicted labels: {'B-PER': 'Marshall, Jason', 'I-PER': 'Faulk, Belser', 'B-ORG': 'Colts, Eagles'}
Data written to data/qa_test_light.csv
Answers: ['B-PER: Kwasniewski, Oscar', 'I-PER: Scalfaro']
Answer: B-PER: Kwasniewski, Oscar
Answer: I-PER: Scalfaro
Answers: ['B-LOC: Ita

In [ ]:
# print('true_labels:', sampled_test_data[3]['ner_tags'])
# print('true_labels:', sampled_test_data[3]['tokens'])
# data = []
# for i, result in enumerate(results):
#     for label, tokens in result.items():
#         #print(label, tokens)
#         tokens = tokens.split(', ')
#         for token in tokens:
#  #           print(token, label, true_labels[i])
#             data.append([token.strip(), label])
# print(data)


# tokens = sampled_test_data[3]['tokens']
# true_labels = sampled_test_data[3]['ner_tags']
# true_label_dict = {label: [] for label in label_mapping.values()}
# for token, true_label in zip(tokens, true_labels):
#     label_str = label_mapping[true_label]
#     if label_str != 'O':
#         true_label_dict[label_str].append(token)

# print(true_label_dict)

true_labels: [0, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 0]
true_labels: ['The', 'West', 'Indies', 'vice-captain', 'struggled', 'for', 'timing', 'during', 'his', '36-minute', 'stay', 'at', 'the', 'crease', 'before', 'chipping', 'a', 'ball', 'from', 'medium', 'pacer', 'Tom', 'Moody', 'straight', 'to', 'Shane', 'Warne', 'at', 'mid-wicket', '.']
[]
{'O': [], 'B-PER': ['Tom', 'Shane'], 'I-PER': ['Moody', 'Warne'], 'B-ORG': [], 'I-ORG': [], 'B-LOC': ['West'], 'I-LOC': ['Indies'], 'B-MISC': [], 'I-MISC': []}


In [ ]:
# import csv
# from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

# # Load true and predicted labels
# true_labels = []
# pred_labels = []

# with open('data/qa_results.csv', newline='') as csvfile:
#     reader = csv.DictReader(csvfile)
#     for row in reader:
#         true_labels.append(row['true'])
#         pred_labels.append(row['pred'])

# # Compute token-level metrics
# print("Precision:", precision_score(true_labels, pred_labels, average='macro', zero_division=0))
# print("Recall:", recall_score(true_labels, pred_labels, average='macro', zero_division=0))
# print("F1 Score:", f1_score(true_labels, pred_labels, average='macro', zero_division=0))

# # Detailed report
# print("\nClassification Report:\n")
# print(classification_report(true_labels, pred_labels, zero_division=0))


Precision: 0.7281962411375407
Recall: 0.8122881731878271
F1 Score: 0.7430039993271146

Classification Report:

              precision    recall  f1-score   support

         LOC       0.74      0.87      0.80        39
        MISC       0.11      0.50      0.18        10
           O       0.99      0.92      0.96       578
         ORG       0.83      0.77      0.80        39
         PER       0.97      1.00      0.98        60

    accuracy                           0.91       726
   macro avg       0.73      0.81      0.74       726
weighted avg       0.96      0.91      0.93       726

